In [1]:
import json
import os
import numpy as np
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import wcs
from astropy.table import Table
import time
import requests
import shutil # save img locally

In [2]:
#Command to retrieve url datas
# RETURNS JPEG IMAGES, FOR VIEWING ONLY
def get_jpgcut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    print(url)
    res = requests.get(url, stream = True)
    file_name = 'Images/Gal{}.jpg'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [3]:
#Command to retrieve url datas for FITS files
def get_fitscut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.fits?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    print(url)
    res = requests.get(url, stream = True)
    file_name = 'Figures/Gal{}.fits'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [4]:
#print table to view data
SGA = Table.read('SGA_MorphsOnly.fits')
SGA

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,HUBBLE_TYPE
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64
284583,SGA-2020 284583,PGC1753576,1753576,246.63893700000006,25.882395000000002,E,83.58,0.4168694,0.61659503,0.049369153,23.009481,16.281,False,LEDA-20181114,102768,2MASXJ16263037+2552589_GROUP,3,False,246.63132430980986,25.886678155342814,0.92581606,2465p260,246.63899443421053,25.88239593752038,1.0317777,SB26,89.25743,0.7660097,246.63894134159338,25.882421271706928,21.809486,7.683644,7.267518,6.908575,6.584644,8.19172,10.043491,12.2072115,14.66538,17.695236,21.416355,25.87598,30.953333,16.50979,15.567412,14.869183,16.380758,15.444356,14.747147,16.275633,15.342991,14.648945,16.190153,15.260059,14.5695095,16.121006,15.193452,14.5059595,16.06036,15.134916,14.4499445,16.00693,15.085361,14.402035,15.962352,15.040784,14.35771,15.920147,15.0014,14.320146,0.030591438,0.04308892,0.045999907,0.04829483,0.06055945,0.080751404,0.0973,0.124484144,0.20583051,0.01532438,0.015401917,0.015267819,0.013852132,0.013975341,0.01385223,0.012736728,0.012889164,0.012807729,0.0118285,0.011991284,0.011952993,0.011088653,0.011258294,0.011255878,0.01050395,0.010681791,0.010707616,0.010012423,0.010216223,0.010258886,0.009653168,0.009853663,0.009897824,0.0092963725,0.009511294,0.009575142,15.662999,7318.958,8.404456e-05,0.7656309,0.11777708,14.756233,4201.656,0.00014010101,0.7699585,0.11079685,14.087339,21311.965,2.6489923e-05,0.7773619,0.118797965,0,-5.0
202259,SGA-2020 202259,PGC1560599,1560599,172.24353299999999,18.4729582,E,49.45,0.41209754,0.57543993,0.16593012,24.64948,17.

In [51]:
#create some pretty pictures
i=0
while i < 10:
    get_jpgcut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    time.sleep(1)
    i+=1

https://www.legacysurvey.org/viewer/cutout.jpg?ra=206.85561899999996&dec=59.754215&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal548100.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=175.20097499999997&dec=33.5209457&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal450699.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=178.11407100000002&dec=34.8944672&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal657041.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=247.95205049999998&dec=19.4742468&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal1325951.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=196.231608&dec=16.582039&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal355862.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=116.1348405&dec=33.0954033&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal8

In [35]:
#collect data for the first i columns
i=0
while i < 100:
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    time.sleep(1)
    i+=1